<a href="https://colab.research.google.com/github/tagasy/training/blob/master/LightgbmTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [0]:
train = pd.read_csv('train.csv')

In [0]:
X = train.drop(['id','target'], axis=1)
y = train["target"].copy()

In [0]:
#評価用関数
def get_evaluate(y_test, predict):

    fpr, tpr, thr_arr = metrics.roc_curve(y_test, predict)

    auc = metrics.auc(fpr, tpr)
    precision = metrics.precision_score(y_test, predict)
    recall = metrics.recall_score(y_test, predict)

    return auc, precision, recall

In [0]:
pip install optuna

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42)
x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train, shuffle=True, random_state=42)

lgb_train = lgb.Dataset(x_tr, y_tr)
lgb_valid = lgb.Dataset(x_val, y_val, reference=lgb_train)

params={
    'objective':  'binary',
    'metrics': 'binary_logloss'
}

model = lgb.train(params,
                  lgb_train,
                  valid_set=lgb_valid,
                  num_boost_round=1000,
                  verbose_eval=10)

y_pred = model.predict(X_test)
predict = [0 if i < 0.5 else 1 for i in y_pred]

auc, precision, recall = get_evaluate(Y_test, predict)
print('AUC:{}, precision:{}, recall:{}'.format(auc, precision, recall))